# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [69]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [70]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,klyuchi,52.2667,79.1667,-2.64,93,100,6.33,RU,1737389560
1,1,edinburgh of the seven seas,-37.0676,-12.3116,18.25,92,100,6.03,SH,1737389562
2,2,pasuquin,18.3328,120.6177,23.17,84,32,6.15,PH,1737389563
3,3,mchinji,-13.7984,32.8802,20.27,83,100,0.62,MW,1737389564
4,4,albany,42.6001,-73.9662,-8.93,65,3,4.47,US,1737389565


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [71]:
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True, 
    size='Humidity',
    alpha=0.7,
    color='#F37021',
    tiles='CartoLight'

)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
)
filtered_df = city_data_df[criteria]


# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,pisco,-13.7000,-76.2167,22.79,78,0,3.60,PE,1737389528
198,198,nouadhibou,20.9310,-17.0347,22.99,49,0,4.12,MR,1737389795
206,206,mar del plata,-38.0023,-57.5575,26.40,69,0,0.89,AR,1737389360
400,400,konarka,19.9000,86.1167,22.06,57,0,3.39,IN,1737390026
456,456,xai-xai,-25.0519,33.6442,26.76,71,0,3.96,MZ,1737390094


### Step 3: Create a new DataFrame called `hotel_df`.

In [73]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

/var/folders/64/v3w6ln215tzd14scrycwkjl40000gn/T/ipykernel_71440/1103971448.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
48,pisco,PE,-13.7000,-76.2167,78,
198,nouadhibou,MR,20.9310,-17.0347,49,
206,mar del plata,AR,-38.0023,-57.5575,69,
400,konarka,IN,19.9000,86.1167,57,
456,xai-xai,MZ,-25.0519,33.6442,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
nouadhibou - nearest hotel: Noadhibou Guesthouse
mar del plata - nearest hotel: Hotel 3 de Abril
konarka - nearest hotel: Hotel Sun Village
xai-xai - nearest hotel: No hotel found
chuhar jamali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,pisco,PE,-13.7000,-76.2167,78,La Portada
198,nouadhibou,MR,20.9310,-17.0347,49,Noadhibou Guesthouse
206,mar del plata,AR,-38.0023,-57.5575,69,Hotel 3 de Abril
400,konarka,IN,19.9000,86.1167,57,Hotel Sun Village
456,xai-xai,MZ,-25.0519,33.6442,71,No hotel found
550,chuhar jamali,PK,24.3895,67.9923,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x='Lng',
    y='Lat', 
    geo=True,
    size='Humidity',
    alpha=0.7,
    color='#F37021',
    tiles='CartoLight',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)